<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/MAGIC_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://archive.ics.uci.edu/dataset/267/banknote+authentication
dataset

In [ ]:
#libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import torch
from torch import nn

In [ ]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [ ]:
#data loading
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fALpha", "fDist", "target"]
df = pd.read_csv("magic04.data",names = cols)
df["target"] = (df["target"] == "g").astype(int)
df = df.sample(frac = 1)
df = df.reset_index(drop = True)

In [ ]:
#preprocessing
y = df.target
X = df.drop(["target"],axis = 1)
X_train ,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.70,random_state = 0)

In [ ]:
#pddf to numpy
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
#assigning number of rows to variables
n = np.shape(X_train)[0]
m = np.shape(X_test)[0]

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [ ]:
X_train = X_train.type(torch.float)
X_test = X_test.type(torch.float)
y_train = y_train.type(torch.float)
y_test = y_test.type(torch.float)

In [ ]:
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [ ]:
class neural(nn.Module):
  def __init__(self, in_size, hidden_size = 8):
    super().__init__()
    self.layer1 = nn.Linear(in_features=in_size,out_features=hidden_size)
    self.layer2 = nn.Linear(in_features=hidden_size,out_features=hidden_size)
    self.layer3 = nn.Linear(in_features=hidden_size,out_features=hidden_size)
    self.layer4 = nn.Linear(in_features=hidden_size,out_features=1)
    self.relu=nn.ReLU()
    self.sigmoid=nn.Sigmoid()
  def forward(self, X):
    return self.sigmoid(self.layer4(self.relu(self.layer3(self.relu(self.layer2(self.relu(self.layer1(X))))))))


In [ ]:
model = neural(in_size = 10, hidden_size = 8).to(device)

In [ ]:
print(model)

neural(
  (layer1): Linear(in_features=10, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (layer3): Linear(in_features=8, out_features=8, bias=True)
  (layer4): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
#torch.manual_seed(42)
epochs = 50000

In [ ]:
for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model(X_train).squeeze()
    y_pred = torch.round(y_logits) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model(X_test).squeeze()
      test_pred = torch.round(test_logits) # logits -> prediction probabilities -> prediction labels
      # 2. Calcuate loss and accuracy
      test_loss = loss_fn(test_logits, y_test)
      test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.63707, Accuracy: 61.41% | Test Loss: 0.62754, Test Accuracy: 68.17%
Epoch: 1000 | Loss: 0.56643, Accuracy: 73.58% | Test Loss: 0.56859, Test Accuracy: 72.82%
Epoch: 2000 | Loss: 0.56193, Accuracy: 73.51% | Test Loss: 0.56391, Test Accuracy: 73.71%
Epoch: 3000 | Loss: 0.55624, Accuracy: 74.52% | Test Loss: 0.55826, Test Accuracy: 76.53%
Epoch: 4000 | Loss: 0.54979, Accuracy: 76.79% | Test Loss: 0.55260, Test Accuracy: 78.86%
Epoch: 5000 | Loss: 0.54554, Accuracy: 77.72% | Test Loss: 0.54966, Test Accuracy: 80.72%
Epoch: 6000 | Loss: 0.54297, Accuracy: 79.25% | Test Loss: 0.54949, Test Accuracy: 79.36%
Epoch: 7000 | Loss: 0.53903, Accuracy: 83.18% | Test Loss: 0.54520, Test Accuracy: 77.85%
Epoch: 8000 | Loss: 0.53758, Accuracy: 80.87% | Test Loss: 0.54483, Test Accuracy: 81.20%
Epoch: 9000 | Loss: 0.53476, Accuracy: 83.58% | Test Loss: 0.54171, Test Accuracy: 79.37%
Epoch: 10000 | Loss: 0.53378, Accuracy: 81.85% | Test Loss: 0.54091, Test Accuracy: 82.00%
Epoch: 11000

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model(X_train).squeeze()
    y_pred = torch.round(y_logits) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model(X_test).squeeze()
      test_pred = torch.round(test_logits) # logits -> prediction probabilities -> prediction labels
      # 2. Calcuate loss and accuracy
      test_loss = loss_fn(test_logits, y_test)
      test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.52722, Accuracy: 83.19% | Test Loss: 0.55252, Test Accuracy: 81.76%
Epoch: 1000 | Loss: 0.52251, Accuracy: 85.19% | Test Loss: 0.53381, Test Accuracy: 81.86%
Epoch: 2000 | Loss: 0.52242, Accuracy: 84.55% | Test Loss: 0.53231, Test Accuracy: 82.79%
Epoch: 3000 | Loss: 0.51615, Accuracy: 86.68% | Test Loss: 0.52579, Test Accuracy: 83.96%
Epoch: 4000 | Loss: 0.51590, Accuracy: 86.91% | Test Loss: 0.52582, Test Accuracy: 83.60%
Epoch: 5000 | Loss: 0.51518, Accuracy: 86.23% | Test Loss: 0.52774, Test Accuracy: 84.17%
Epoch: 6000 | Loss: 0.51366, Accuracy: 86.91% | Test Loss: 0.52622, Test Accuracy: 83.81%
Epoch: 7000 | Loss: 0.51350, Accuracy: 87.09% | Test Loss: 0.52633, Test Accuracy: 83.86%
Epoch: 8000 | Loss: 0.51356, Accuracy: 86.60% | Test Loss: 0.52609, Test Accuracy: 83.81%
Epoch: 9000 | Loss: 0.51466, Accuracy: 86.15% | Test Loss: 0.52549, Test Accuracy: 84.23%
Epoch: 10000 | Loss: 0.51400, Accuracy: 87.15% | Test Loss: 0.52643, Test Accuracy: 83.86%
Epoch: 11000